In [8]:
import os
import sys
import numpy as np
import pandas as pd
import re

### SPEARMINT: Predictive Entropy Search

Parse-Experiments

In [9]:
def read_tail(f, lines=1, buffer_size=4098):
    lines_found = []
    block_counter = -1
    while len(lines_found) < lines:
        try:
            f.seek(block_counter * buffer_size, os.SEEK_END)
        except IOError:
            f.seek(0)
            lines_found = f.readlines()
            break

        lines_found = f.readlines()
        block_counter -= 1

    return lines_found[-lines:]

In [10]:
def parse_experiment(exp_dir, range_min, range_max, use_dist=False):
    results = pd.DataFrame(columns=['iteration', 'pure_loss'])
    if use_dist:
        results['dist'] = None
    output_dir = os.path.join(exp_dir, 'output')
    for i in range(range_min, range_max+1):
        file_name = "{:08d}".format(i) + '.out'
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, 'rb') as f:
            lines = read_tail(f, lines=2)
        if use_dist:
            cur_r = re.findall(r"[-+]?\d*\.\d+|\d+", lines[0].decode("utf-8").strip())[0]
            cur_d = re.findall(r"[-+]?\d*\.\d+|\d+", lines[1].decode("utf-8").strip())[0]
            results = results.append({'iteration':i, 'pure_loss':float(cur_r), 'dist':float(cur_d)},
                                     ignore_index=True)
        else:
            cur_r = re.findall(r"[-+]?\d*\.\d+|\d+", lines[1].decode("utf-8").strip())[0]
            results = results.append({'iteration':i, 'pure_loss':float(cur_r)},
                                     ignore_index=True)
    results['iteration'] = results['iteration'].astype('int')
    results = results.set_index('iteration')
    return results

In [11]:
sorted(os.listdir('../Spearmint-PESM/experiments_PES/'))

['6hump_nf',
 '6hump_noisy',
 'MNIST_SVM',
 'branin_nf',
 'branin_noisy',
 'griewank6_nf',
 'griewank6_noisy',
 'rosenbrock_nf',
 'rosenbrock_noisy']

In [12]:
save_names = [
 'hump_nf',
 'hump_noisy',
 'MNIST_SVM',
 'branin_nf',
 'branin_noisy',
 'griewank_nf',
 'griewank_noisy',
 'rosenbrock_nf',
 'rosenbrock_noisy']

In [14]:
out_dir = os.path.join('results')
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)


for exp_name, save_name in zip(sorted(os.listdir('../Spearmint-PESM/experiments_PES/')), save_names):
    dist = 'SVM' not in exp_name
    full_name = os.path.join('../Spearmint-PESM/experiments_PES/', exp_name)
    df = parse_experiment(full_name, 1, 40, dist)
    df.to_csv(os.path.join(out_dir, save_name)+'.csv')